In [14]:
import numpy as np
import pandas as pd
import yfinance as yf

import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

## Data Preparation Notebook

### OHLC Dataframes

In [15]:
start = '2019-01-01'
end = '2024-04-01'

## Top 5 Most Traded Forex Pairs in the World (Source: https://www.ig.com/en/trading-strategies/top-10-most-traded-currency-pairs-191206)
pairs = ['EURUSD=X', 'GBPUSD=X', 'CHFUSD=X', 'CADUSD=X']

## Name of the future .csv file that will contain the historical ohlc dataframe of each pair.
data_names = ['eurusd.csv', 'gbpusd.csv', 'chfusd.csv', 'cadusd.csv']


## Import the historical ohlc dataframe of each pair from Yahoo Finance and save it in the data folder as a .csv
for i in range(4):
    df = yf.download(pairs[i], start=start, end=end, progress=False).iloc[:,:4]
    df.to_csv(f'data/ohlc_df/{data_names[i]}')

### Technical Indicators

In [16]:
def technical_indicators(data):
    '''
    This function computes the following technical indicators to a OHLC dataframe:
    * Moving Average (MA)
    * Exponential Moving Average (EMA)
    * Moving Average Convergence/Divergence (MACD)
    * Bollinger Bands
    * Average True Range (ATR)
    * Relative Strength Index (RSI)
    * Pivot Point
    * Commodity Channel Index
    * Momentum
    '''
    ## MA
    data['MA10'] = data.Close.rolling(10).mean()

    ## EMA
    data['EMA12'] = data.Close.ewm(span=12, adjust=False).mean()
    data['EMA26'] = data.Close.ewm(span=26, adjust=False).mean()

    ## MACD
    data['MACD'] = data['EMA12'] - data['EMA26']

    ## Bollinger Bands
    rolling_mean = data.Close.rolling(window=20).mean()
    rolling_std = data.Close.rolling(window=20).std()

    data['UB'] = rolling_mean + 2*rolling_std
    data['LB'] = rolling_mean - 2*rolling_std
    
    ## Average True Range (ATR)
    high_low = data.High - data.Low
    highprevclose = abs(data.High - data.Close.shift(1))
    lowprevclose = abs(data.Low - data.Close.shift(1))
    true_range = pd.concat([high_low, highprevclose, lowprevclose], axis=1).max(axis=1)
    
    data['ATR'] = true_range.rolling(window=14).mean()
    
    ## Relative Strength Index (RSI)
    delta = data.Close.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=10).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=10).mean()
    rs = gain / loss
    
    data['RSI'] = 100 - (100 / (1 + rs))

    ## Pivot Point & Support/Resistances
    data['pivot'] = (data.High + data.Low + data.Close) / 3
    data['s1'] = data['pivot']*2 - data.High
    data['r1'] = data['pivot']*2 - data.Low

    data['s2'] = data['pivot'] - (data.High - data.Low)
    data['r2'] = data['pivot'] + (data.High - data.Low)

    data['s3'] = data.Low - 2*(data.High - data['pivot'])
    data['r3'] = data.High + 2*(data['pivot'] - data.Low)

    ## Commodity Channel Index
    typical_price = (df['Close'] + df['Low'] + df['High'])/3
    mean_deviation = abs(typical_price - typical_price.rolling(window=20).mean())
    mean_deviation_sum = mean_deviation.rolling(window=20).sum()

    data['CCI'] = (typical_price - typical_price.rolling(window=20).mean()) / (0.015 * mean_deviation_sum)
    
    ## Momentum
    data['mom'] = data.Close.diff(4)

    return data
    

In [17]:
## Create the .csv with the ohlc dataframe and the technical indicators for each pairs.
data_names_tech = ['eurusd_tech.csv', 'gbpusd_tech.csv', 'chfusd_tech.csv', 'cadusd_tech.csv']

for i in range(4):
    data = pd.read_csv(f'data/ohlc_df/{data_names[i]}', index_col=0)
    data = technical_indicators(data).iloc[:,3:].dropna()
    data.to_csv(f'data/technical_df/{data_names_tech[i]}')

### Macroeconomic Indicators

In [18]:
def import_macrodata(filename='cad_inflation.csv'):
    '''
    This function imports the .csv file according to the parameters and transform it into a daily dataframe.
    '''
    type = filename.split('.')[0].split('_')[1]
    if type == 'inflation':
        df1 = pd.read_csv(f'data/inflation_rates/{filename}', index_col=1).iloc[:,1]
    elif type == 'interest':
        df1 = pd.read_csv(f'data/interest_rates/{filename}', index_col=1).iloc[:,1]
    else:
        raise ValueError("This type of file doesn't exist.")
    
    df1.index = pd.to_datetime(df1.index)
    df2 = pd.Series(index=pd.date_range(start, end))
    df2.update(df1)
    df2 = df2.ffill()

    if pd.isna(df2[0]):
        df2[0] = df1[df1.index.year == df2.index[0].year - 1].iloc[-1]
        df2 = df2.ffill()
    
    return pd.DataFrame(df2, columns=[filename.split('.')[0]])

In [1]:
def macroeconomic_indicators(pair):
    '''
    This functions concats the OHLC dataframe of a pair with the major index of the countries related to the pair.
    It also imports the interest and inflation rates.
    '''
    data_name = f"{pair.split('=')[0].lower()}.csv"
    ## Import the dataframe
    data = pd.read_csv(f'data/ohlc_df/{data_name}', index_col=0).iloc[:,3]
    data.index = pd.to_datetime(data.index)

    ## Add the US major index (Close Price)
    major_index = ['^GSPC', '^DJI', '^IXIC']

    usa_interest = import_macrodata(filename='usd_interest.csv')
    usa_inflation = import_macrodata(filename='usd_inflation.csv')

    ## Add the non-US major index (Close Price)
    if pair == 'EURUSD=X':
        other_index = ['^GDAXI', '^FCHI', 'FTSEMIB.MI']

        df_interest = import_macrodata(filename='eur_interest.csv')
        df_inflation = import_macrodata(filename='eur_inflation.csv')

    elif pair == 'GBPUSD=X':
        other_index = ['^FTSE']

        df_interest = import_macrodata(filename='gbp_interest.csv')
        df_inflation = import_macrodata(filename='gbp_inflation.csv')

    elif pair == 'CHFUSD=X':
        other_index = ['^SSMI']

        df_interest = import_macrodata(filename='chf_interest.csv')
        df_inflation = import_macrodata(filename='chf_inflation.csv')

    elif pair == 'CADUSD=X':
        other_index = ['^GSPTSE']

        df_interest = import_macrodata(filename='cad_interest.csv')
        df_inflation = import_macrodata(filename='cad_inflation.csv')
        
    else:
        raise ValueError('Pair is not supported.')

    major_index += other_index
    indexes = yf.download(major_index, start=start, end=end, progress=False).Close
    data = pd.concat([data, indexes, usa_interest, usa_inflation, df_interest, df_inflation], axis=1).dropna()

    return data

In [20]:
data_names_macro = ['eurusd_macro.csv', 'gbpusd_macro.csv', 'chfusd_macro.csv', 'cadusd_macro.csv']

for i in range(4):
    data = macroeconomic_indicators(pair=pairs[i])
    data.to_csv(f'data/macro_df/{data_names_macro[i]}')

### Technical & Macroeconomic Indicators

In [ ]:
## We make dataframes with both technical and macroeconomic indicators.

data_names_both = ['eurusd_both.csv', 'gbpusd_both.csv', 'chfusd_both.csv', 'cadusd_both.csv']

for i in range(4):
    df2 = pd.read_csv(f'data/macro_df/{data_names_macro[i]}', index_col=0).iloc[:,4:]
    df1 = pd.read_csv(f'data/technical_df/{data_names_tech[i]}', index_col=0)
    data = pd.concat([df1, df2], axis=1).dropna()
    data.to_csv(f'data/both_df/{data_names_both[i]}')